In [30]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
import pickle
import gensim.downloader as api
import gensim
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import contractions
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
torch.manual_seed(25)

[nltk_data] Downloading package wordnet to C:\Users\Abhinav
[nltk_data]     Jindal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Data

Reading data from data.tsv file in the current diectory, using separator as \t and skipping bad data lines

In [2]:
raw_dataset = pd.read_csv(
     './data.tsv', 
     sep='\t',
     on_bad_lines='skip'
)

C:\Users\Abhinav Jindal\AppData\Local\Temp\ipykernel_4396\3209690956.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_dataset = pd.read_csv(


## Keep reviews and star rating

only keeping the "review body" and "star rating" columns in the read dataset

In [3]:
filtered_dataset = raw_dataset[['review_body', 'star_rating']]
filtered_dataset['review_body'] = filtered_dataset['review_body'].astype('str', errors='ignore')
filtered_dataset['star_rating'] = filtered_dataset['star_rating'].astype('int64', errors='ignore')

C:\Users\Abhinav Jindal\AppData\Local\Temp\ipykernel_4396\3482623344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset['review_body'] = filtered_dataset['review_body'].astype('str', errors='ignore')
C:\Users\Abhinav Jindal\AppData\Local\Temp\ipykernel_4396\3482623344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset['star_rating'] = filtered_dataset['star_rating'].astype('int64', errors='ignore')


## We form three classes and select 20000 reviews randomly from each class.

We create 3 classes similar to HW1 where ratings <= 2 and given class 1, greater than equal to 4 are given class3 and rest are given class 2. We then sample 20000 random samples from each class to create our dataset.

In [4]:
def class_assign(value):
    try:
        value = float(value)
        if value <= 2:
            return 1
        elif value >= 4:
            return 3
        else:
            return 2
    except Exception as e:
        return 4
 
filtered_dataset['class'] = filtered_dataset['star_rating'].map(class_assign)
df1 = filtered_dataset.loc[filtered_dataset['class'] == 1].sample(20000, random_state=30)
df2 = filtered_dataset.loc[filtered_dataset['class'] == 2].sample(20000, random_state=30)
df3 = filtered_dataset.loc[filtered_dataset['class'] == 3].sample(20000, random_state=30)
dataset = pd.concat([df1, df2, df3])
dataset = dataset.sample(frac=1).reset_index(drop=True)

C:\Users\Abhinav Jindal\AppData\Local\Temp\ipykernel_4396\808810635.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset['class'] = filtered_dataset['star_rating'].map(class_assign)


### Used pickle for writing and reading some objects to reduce some computational load while testing and finetuning parameters.

In [5]:
def write_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)                
        f.close()

def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)                     
        f.close()
        return obj

### write and read the dataset from pickle for faster computations


In [8]:
write_pickle(dataset, './dataset.pkl')

In [31]:
dataset = load_pickle('./dataset.pkl')
dataset

,review_body,star_rating,class
0,great product,5.0,3
1,My wife received this hair dryer as a gift a f...,2,1
2,How do you compare the hundreds of skin care p...,4,3
3,Every well groomed guy needs one of these! My ...,5,3
4,Boy is this stuff hot!,5,3
...,...,...,...
59995,I love this nail polish! All you do is put on ...,5,3
59996,"I am allergic to everything; however, I can us...",5,3
59997,So far I haven't seen anything about this eye ...,3,2
59998,Not worth the money for 100% plastic. No linin...,1,1


In [32]:
def clean_review_body(s):
    s = s.lower()
    # remove html tags
    s = re.sub(r'<[^<]+?>', ' ', s)
    # remove urls
    s = re.sub(r'http\S+', ' ', s)
    s = re.sub(r'www\S+', ' ', s)
    # remove extra spaces
    s = re.sub(r"\s+", ' ', s)
    # fix contractions
    s = contractions.fix(s)
    # remove non word characters
    s = re.sub(r"[^a-z\s]", ' ', s)
    return s

dataset['review_body'] = dataset['review_body'].map(clean_review_body)

## Task 2: Word Embeddings

## Part (a): word2vec-google-news-300

Loaded the google news 300 word2vec model and created 2 helper functions, one to get the vec for a word and handle exceptions if any and, other to get similarity score between 2 vectors.

In [34]:
try:
    # reading the model from pickle if it's already ran once and saved using pickle
    google_model = load_pickle('./google_model.pkl')
except Exception as e:
    # loading the google news word2vec model  and saving it using pickle if not already present to improve computational load
    google_model = api.load('word2vec-google-news-300')
    write_pickle(google_model, './google_model.pkl')

def get_word_to_vec(word, model):
    try:
        vec = model[word]
    except KeyError:
        print("The word '{}' does not appear in this model".format(word))
        vec = None
    return vec

def get_similarity(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

In [29]:
king_vec = get_word_to_vec('king', google_model)
man_vec = get_word_to_vec('man', google_model)
woman_vec = get_word_to_vec('woman', google_model)
queen_vec = get_word_to_vec('queen', google_model)
approx_queen_vec = king_vec - man_vec + woman_vec
print("Google word2vec similarity scores")
print ("Similarity between queen and (king - man + woman):", get_similarity(queen_vec, approx_queen_vec))
print ("Similarity between jacket and coat:", google_model.similarity('jacket', 'coat'))
print ("Similarity between king and france:", google_model.similarity('king', 'france'))

Google word2vec similarity scores
Similarity between queen and (king - man + woman): 0.73005176
Similarity between jacket and coat: 0.6492949
Similarity between king and france: 0.16112953


## Part (b): Train word2vec using own dataset

training custom word2vec model using our dataset with window size 13 and vector size 300.

In [18]:
## applying preprocessing provided by gensim library
review_text = dataset['review_body'].apply(gensim.utils.simple_preprocess)
model = gensim.models.Word2Vec(
    window=13,
    vector_size=300,
    min_count=9,
    workers=4,
)
model.build_vocab(review_text)
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

model.save("./word2vec-custom.model")

In [28]:
model = gensim.models.Word2Vec.load("./word2vec-custom.model")
king_vec = get_word_to_vec('king', model.wv)
man_vec = get_word_to_vec('man', model.wv)
woman_vec = get_word_to_vec('woman', model.wv)
queen_vec = get_word_to_vec('queen', model.wv)
approx_queen_vec = king_vec - man_vec + woman_vec
print("Our custom word embedding model similarity scores")
print ("Similarity between queen and (king - man + woman):", get_similarity(queen_vec, approx_queen_vec))
print ("Similarity between jacket and coat:", model.wv.similarity('jacket', 'coat'))
print ("Similarity between king and france:", model.wv.similarity('king', 'france'))

Our custom word embedding model similarity scores
Similarity between queen and (king - man + woman): 0.32972342
Similarity between jacket and coat: 0.041718163
Similarity between king and france: 0.57025325


#### The embeddings generated by the pretrained google word2vec seems to be much better compared to the model trained on our own dataset.  The pretrained google model encodes semantic similarities between words better.  This is probably because of the difference in the amount of data used to train these models. Google model is trained on a large dataset while our model just uses 60k reviews.

##### From our examples we see, it assigns higher score of 0.73 to similarity between woman and (king-man+woman) compared to 0.33 in our custom dataset. Also, jacket and coat are given a similarity score of 0.65 in google model whereas our model gives it a small score of 0.04 even though jacket and coat are highly similar and are often used in the same context of clothes. Similarly for very dissimilar words like king and france, google model gives a lower score of 0.16 and our model gives 0.57 which is not appropriate as these 2 words are not that highly related and are rarely used in the same context. Hence, google pretrained model returns much better encoding compared to our model. This could be primarily because google model is trained on a very large dataset compared to our model which is just trained on 60k reviews which is not a lot of data for a good word2vec model.

## Task 3

#### we use gensims simple preprocess to do further minor preprocessing already provided by gensim to improve our predictions

In [35]:
dataset['review_preprocessed_tokens'] = dataset['review_body'].apply(gensim.utils.simple_preprocess)

#### we calculate the mean vectors from the embeddings by google model, if none of the words are present in the google model we return a vector of zeros of size 300

#### we then split the dataset and train models on our training dataset and test on testing dataset

In [51]:
def mean_vector(review_preprocessed_tokens):
    vectors = [google_model[word] for word in review_preprocessed_tokens if word in google_model]
    if len(vectors) > 0:
        feature_vec = np.mean(vectors, axis=0)
    else:
        feature_vec = np.zeros(300)
    return feature_vec

dataset['reviews_vector'] = dataset['review_preprocessed_tokens'].map(mean_vector)
finished_dataset = dataset[['reviews_vector', 'class']]

training_data, testing_data = train_test_split(finished_dataset, test_size=0.2, random_state=25)

train_X = np.stack(training_data['reviews_vector'])
train_Y = np.array(training_data['class'])
test_X = np.stack(testing_data['reviews_vector'])
test_Y =np.array(testing_data['class'])

In [38]:
clf = Perceptron(penalty='elasticnet', alpha=0.00001, tol=1e-7, random_state=10)
clf.fit(train_X, train_Y)
print ("Accuracy for word2vec perceptron:", accuracy_score(test_Y, clf.predict(test_X)))

Accuracy for word2vec perceptron: 0.59025


Accuracy using tf-idf for perceptron (calculated from HW1): 0.6474166666666666

In [39]:
clf = LinearSVC(penalty='l2', loss='squared_hinge', tol=1e-7, dual=True, random_state=25)
clf.fit(train_X, train_Y)
print ("Accuracy for word2vec SVM:", accuracy_score(test_Y, clf.predict(test_X)))

Accuracy for word2vec SVM: 0.6615833333333333


Accuracy using tf-idf for SVM (calculated from HW1): 0.7070833333333333

### We see that tf-idf performs better for these 2 models, perceptron and SVM when compared with word2vec. This could be because word2vec vectors have complicated relations which are not easily captured by these simple models. Also, taking mean of the word2vec vectors might lead to loss of information which could be another cause of lesser accuracy 

## Task 4

In [40]:
## change to cuda if want to use GPU
device = "cpu"

In [41]:
## move data to tensors and appropriate device and also change label to torch conventions
def format_data_for_model(train_X, train_Y, test_X, test_Y, device):
    X_train = torch.tensor(train_X).to(torch.float)
    X_test = torch.tensor(test_X).to(torch.float)
    Y_train = torch.tensor(train_Y).long()
    Y_test = torch.tensor(test_Y).long()
    X_train = X_train.to(device)
    Y_train = Y_train.to(device)
    X_test = X_test.to(device)
    Y_test = Y_test.to(device)
    # move labels from 1,2,3 to 0,1,2 to fit to pytorch conventions
    Y_train = Y_train - 1
    Y_test = Y_test - 1
    return X_train, X_test, Y_train, Y_test

In [42]:
## trains model using parameters passed and prints train and test loss and accuracy after each epoch
def train_model(model, optimizer, loss_fn, X_train, Y_train, X_test, Y_test, num_epochs, batch_size):
    for epoch in range(num_epochs):
        model.train()

        # Shuffle the training data
        indices = torch.randperm(X_train.shape[0])
        x_train = X_train[indices]
        y_train = Y_train[indices]

        # batch for training data
        for i in range(0, x_train.shape[0], batch_size):
            X_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]

            # Compute the forward pass through the network
            y_pred = model(X_batch)                
            
            ## zero out the gradient, calcualate loss and back propagate loss
            optimizer.zero_grad()
            loss = loss_fn(y_pred, y_batch)
            loss.backward()

            # Update the model parameters
            optimizer.step()

        # Evaluate performance after each epoch
        with torch.no_grad():
            model.eval()

            # train loss and accuracy
            y_pred = model(X_train)
            train_loss = loss_fn(y_pred, Y_train)
            y_pred = torch.argmax(y_pred, dim=1)
            train_accuracy = (y_pred == Y_train).float().mean()

            # test loss and accuracy
            y_pred = model(X_test)
            test_loss = loss_fn(y_pred, Y_test)
            y_pred = torch.argmax(y_pred, dim=1)
            test_accuracy = (y_pred == Y_test).float().mean()

        # Print the epoch, loss, and accuracy
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    return model


## Task 4(a)  MLP using average Word2Vec vectors

#### We want to use the same mean vectors data we used for simple models, hence no separate preprocessing

In [43]:
X_train, X_test, Y_train, Y_test = format_data_for_model(train_X, train_Y, test_X, test_Y, device)

#### We create a MLP model class with 2 hidden layers (100 and 10 nodes each) and use Relu as the activation function. We also use SGD optimizer and CrossEntropyLoss as this was giving the best accuracy.

In [50]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(300, 100)
        self.fc2 = nn.Linear(100, 10)
        self.fc3 = nn.Linear(10, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Create the MLP model and optimizer
model = MLP()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# As the loss is CrossEntropy we don't need to add softmax separately
loss_fn = nn.CrossEntropyLoss()
model.to(device)

MLP(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (relu): ReLU()
)

I have used the following hyperparameters: <br/>
learning rate = 0.01 <br/>
loss function = CrossEntropyLoss <br/>
optimizer = SGD <br/>
momentum = 0.9 <br/>
non-linear activation = Relu <br/>
epochs = 50 <br/>
batch size = 32

In [45]:
## trains the model and prints accuracy after each epoch
model = train_model(model, optimizer, loss_fn, X_train, Y_train, X_test, Y_test, num_epochs=50, batch_size=32)

Epoch 1/50, Train Loss: 0.8410, Train Accuracy: 0.6064
Epoch 1/50, Test Loss: 0.8459, Test Accuracy: 0.6040
Epoch 2/50, Train Loss: 0.7828, Train Accuracy: 0.6531
Epoch 2/50, Test Loss: 0.7882, Test Accuracy: 0.6518
Epoch 3/50, Train Loss: 0.7750, Train Accuracy: 0.6447
Epoch 3/50, Test Loss: 0.7869, Test Accuracy: 0.6363
Epoch 4/50, Train Loss: 0.7711, Train Accuracy: 0.6522
Epoch 4/50, Test Loss: 0.7830, Test Accuracy: 0.6495
Epoch 5/50, Train Loss: 0.7504, Train Accuracy: 0.6685
Epoch 5/50, Test Loss: 0.7641, Test Accuracy: 0.6602
Epoch 6/50, Train Loss: 0.7992, Train Accuracy: 0.6382
Epoch 6/50, Test Loss: 0.8207, Test Accuracy: 0.6277
Epoch 7/50, Train Loss: 0.7520, Train Accuracy: 0.6643
Epoch 7/50, Test Loss: 0.7749, Test Accuracy: 0.6511
Epoch 8/50, Train Loss: 0.7308, Train Accuracy: 0.6803
Epoch 8/50, Test Loss: 0.7498, Test Accuracy: 0.6712
Epoch 9/50, Train Loss: 0.7390, Train Accuracy: 0.6708
Epoch 9/50, Test Loss: 0.7643, Test Accuracy: 0.6584
Epoch 10/50, Train Loss: 0.7

In [47]:
print (f"Accuracy for MLP for mean word2vec: {accuracy_score(Y_test, torch.argmax(model(X_test), dim=1)):.4f}")

Accuracy for MLP for mean word2vec: 0.6717


In [52]:
## deleting these variables to improve memory load
del model, train_X, train_Y, test_X, test_Y, X_train, X_test, Y_train, Y_test

## Task 4(b)  concatenate the first 10 Word2Vec vectors for each review as the input feature

#### For concatenation we use ignore the words that are not present in google model and if the number of valid words are less than 10, we pad the rest of the vector with zeros

#### We create a MLP model class with 2 hidden layers (100 and 10 nodes each) and use Relu as the activation function. We also use SGD optimizer and CrossEntropyLoss as this was giving the best accuracy.

In [54]:
def concatenated_vector(words):
    num_vectors = 0
    i = 0
    vector = np.empty((10*300))
    while num_vectors < 10:
        if i < len(words):
            current_vector = google_model[words[i]] if words[i] in google_model else None
            i += 1
            if current_vector is None:
                continue
        else:
            current_vector = np.zeros((300))
        vector[num_vectors*300: (num_vectors+1)*300] = current_vector
        num_vectors += 1
    return vector

dataset['reviews_vector'] = dataset['review_preprocessed_tokens'].map(concatenated_vector)
finished_dataset = dataset[['reviews_vector', 'class']]

training_data, testing_data = train_test_split(finished_dataset, test_size=0.2, random_state=25)

train_X = np.stack(training_data['reviews_vector'])
train_Y = np.array(training_data['class'])
test_X = np.stack(testing_data['reviews_vector'])
test_Y = np.array(testing_data['class'])

## have data as tensors
X_train, X_test, Y_train, Y_test = format_data_for_model(train_X, train_Y, test_X, test_Y, device)

In [57]:
# Define the multilayer perceptron network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(300*10, 100)
        self.fc2 = nn.Linear(100, 10)
        self.fc3 = nn.Linear(10, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Create the MLP model, optimizer and loss function
model = MLP()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
model.to(device)

MLP(
  (fc1): Linear(in_features=3000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (relu): ReLU()
)

I have used the following hyperparameters: <br/>
learning rate = 0.001 <br/>
loss function = CrossEntropyLoss <br/>
optimizer = SGD <br/>
momentum = 0.9 <br/>
non-linear activation = Relu <br/>
epochs = 20 <br/>
batch size = 32

In [58]:
model = train_model(model, optimizer, loss_fn, X_train, Y_train, X_test, Y_test, num_epochs=20, batch_size=32)

Epoch 1/20, Train Loss: 1.0758, Train Accuracy: 0.4897
Epoch 1/20, Test Loss: 1.0760, Test Accuracy: 0.4869
Epoch 2/20, Train Loss: 0.9562, Train Accuracy: 0.5220
Epoch 2/20, Test Loss: 0.9591, Test Accuracy: 0.5193
Epoch 3/20, Train Loss: 0.9066, Train Accuracy: 0.5654
Epoch 3/20, Test Loss: 0.9209, Test Accuracy: 0.5518
Epoch 4/20, Train Loss: 0.8707, Train Accuracy: 0.5950
Epoch 4/20, Test Loss: 0.8978, Test Accuracy: 0.5739
Epoch 5/20, Train Loss: 0.8460, Train Accuracy: 0.6092
Epoch 5/20, Test Loss: 0.8853, Test Accuracy: 0.5823
Epoch 6/20, Train Loss: 0.8235, Train Accuracy: 0.6217
Epoch 6/20, Test Loss: 0.8747, Test Accuracy: 0.5918
Epoch 7/20, Train Loss: 0.8080, Train Accuracy: 0.6319
Epoch 7/20, Test Loss: 0.8720, Test Accuracy: 0.5928
Epoch 8/20, Train Loss: 0.7924, Train Accuracy: 0.6418
Epoch 8/20, Test Loss: 0.8683, Test Accuracy: 0.5901
Epoch 9/20, Train Loss: 0.7752, Train Accuracy: 0.6497
Epoch 9/20, Test Loss: 0.8674, Test Accuracy: 0.5932
Epoch 10/20, Train Loss: 0.7

In [59]:
print (f"Accuracy for MLP for concatenated word2vec: {accuracy_score(Y_test, torch.argmax(model(X_test), dim=1)):.4f}")

Accuracy for MLP for concatenated word2vec: 0.5791


In [60]:
del model, X_train, X_test, Y_train, Y_test

#### The accuracy using mean word2vec is greater than concatenated word2vec on testing set. This could be because we are just considering the first 10 words in a review which might not contain a lot of useful information  many times.

#### When compared with simple models with word2vec embeddings (Task 3), we see that MLP with mean vectors performs better than Perceptron and SVM. This is probably because it is able to encode complex information in the vectors more effectively when comapred to the simple models. MLP with concatenated vectors still performs worse. This is probably because of the same reason as mentioned above that taking first 10 words might be leading to loss of information present in the rest of the review.

# Task 5

#### For serial vectors, we consider the first 20 valid words present in google model in the review and pad the remaining vectors (if 20 valid vectors are not found) with zeros. Thus we get a 20*300 input vector for each instance.

In [61]:
def series_vector(words):
    num_vectors = 0
    i = 0
    vector = np.empty((20,300))
    while num_vectors < 20:
        if i < len(words):
            current_vector = google_model[words[i]] if words[i] in google_model else None
            i += 1
            if current_vector is None:
                continue
        else:
            current_vector = np.zeros((300))
        vector[num_vectors] = current_vector
        num_vectors += 1
    return vector

dataset['reviews_vector'] = dataset['review_preprocessed_tokens'].map(series_vector)
finished_dataset = dataset[['reviews_vector', 'class']]

training_data, testing_data = train_test_split(finished_dataset, test_size=0.2, random_state=25)

train_X = np.stack(training_data['reviews_vector'])
train_Y = np.array(training_data['class'])
test_X = np.stack(testing_data['reviews_vector'])
test_Y =np.array(testing_data['class'])

X_train, X_test, Y_train, Y_test = format_data_for_model(train_X, train_Y, test_X, test_Y, device)

## Task 5 (a) RNN

#### We create a RNN model class with a hidden layer of size 20. The output of the hidden layer (20 outputs from each series) is averaged before passing to the FC later during forward pass. We also use SGD optimizer and CrossEntropyLoss as this was giving the best accuracy.

In [62]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        output, hidden = self.rnn(text)
        output = output.mean(dim=1)
        fc_output = self.fc(output)
        return fc_output

model = RNN(300, 20, 3)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
model.to(device)

RNN(
  (rnn): RNN(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=3, bias=True)
)

I have used the following hyperparameters: <br/>
learning rate = 0.01 <br/>
loss function = CrossEntropyLoss <br/>
optimizer = SGD <br/>
momentum = 0.9 <br/>
epochs = 40 <br/>
batch size = 32

In [63]:
model = train_model(model, optimizer, loss_fn, X_train, Y_train, X_test, Y_test, num_epochs=40, batch_size=32)

Epoch 1/40, Train Loss: 0.9357, Train Accuracy: 0.5101
Epoch 1/40, Test Loss: 0.9421, Test Accuracy: 0.5083
Epoch 2/40, Train Loss: 0.8597, Train Accuracy: 0.6046
Epoch 2/40, Test Loss: 0.8688, Test Accuracy: 0.6024
Epoch 3/40, Train Loss: 0.8344, Train Accuracy: 0.6231
Epoch 3/40, Test Loss: 0.8428, Test Accuracy: 0.6167
Epoch 4/40, Train Loss: 0.8351, Train Accuracy: 0.6244
Epoch 4/40, Test Loss: 0.8460, Test Accuracy: 0.6168
Epoch 5/40, Train Loss: 0.8035, Train Accuracy: 0.6359
Epoch 5/40, Test Loss: 0.8181, Test Accuracy: 0.6252
Epoch 6/40, Train Loss: 0.7936, Train Accuracy: 0.6407
Epoch 6/40, Test Loss: 0.8079, Test Accuracy: 0.6327
Epoch 7/40, Train Loss: 0.8058, Train Accuracy: 0.6293
Epoch 7/40, Test Loss: 0.8182, Test Accuracy: 0.6198
Epoch 8/40, Train Loss: 0.8001, Train Accuracy: 0.6373
Epoch 8/40, Test Loss: 0.8203, Test Accuracy: 0.6289
Epoch 9/40, Train Loss: 0.7878, Train Accuracy: 0.6453
Epoch 9/40, Test Loss: 0.8127, Test Accuracy: 0.6347
Epoch 10/40, Train Loss: 0.7

In [65]:
print (f"Accuracy for RNN: {accuracy_score(Y_test, torch.argmax(model(X_test), dim=1)):.4f}")

Accuracy for RNN: 0.6478


#### The accuracy is slightly less than MLP with mean word2vec vectors but it is higher than the MLP with concatenated word2vec vectors. This is because in RNN we are considering the first 20 words which have more information than 10 words in concatenated word2vec. Also, MLP with mean vectors might be performing better because of the vanishing gradient problem in RNN and that still some information might be missing for reviews greater than length 20. 

## Task 5 (b) GRU

#### same as RNN model but GRU layer instead of RNN

In [66]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        output, hidden = self.rnn(text)
        output = output.mean(dim=1)
        fc_output = self.fc(output)
        return fc_output

model = GRU(300, 20, 3)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
model.to(device)

GRU(
  (rnn): GRU(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=3, bias=True)
)

I have used the following hyperparameters: <br/>
learning rate = 0.01 <br/>
loss function = CrossEntropyLoss <br/>
optimizer = SGD <br/>
momentum = 0.9 <br/>
epochs = 30 <br/>
batch size = 32

In [67]:
model = train_model(model, optimizer, loss_fn, X_train, Y_train, X_test, Y_test, num_epochs=30, batch_size=32)

Epoch 1/30, Train Loss: 0.8834, Train Accuracy: 0.5820
Epoch 1/30, Test Loss: 0.8865, Test Accuracy: 0.5762
Epoch 2/30, Train Loss: 0.8301, Train Accuracy: 0.6193
Epoch 2/30, Test Loss: 0.8340, Test Accuracy: 0.6189
Epoch 3/30, Train Loss: 0.8181, Train Accuracy: 0.6231
Epoch 3/30, Test Loss: 0.8221, Test Accuracy: 0.6220
Epoch 4/30, Train Loss: 0.7996, Train Accuracy: 0.6370
Epoch 4/30, Test Loss: 0.8061, Test Accuracy: 0.6348
Epoch 5/30, Train Loss: 0.7844, Train Accuracy: 0.6437
Epoch 5/30, Test Loss: 0.7954, Test Accuracy: 0.6352
Epoch 6/30, Train Loss: 0.7980, Train Accuracy: 0.6357
Epoch 6/30, Test Loss: 0.8120, Test Accuracy: 0.6268
Epoch 7/30, Train Loss: 0.7885, Train Accuracy: 0.6415
Epoch 7/30, Test Loss: 0.8021, Test Accuracy: 0.6342
Epoch 8/30, Train Loss: 0.7570, Train Accuracy: 0.6581
Epoch 8/30, Test Loss: 0.7743, Test Accuracy: 0.6455
Epoch 9/30, Train Loss: 0.7557, Train Accuracy: 0.6581
Epoch 9/30, Test Loss: 0.7759, Test Accuracy: 0.6445
Epoch 10/30, Train Loss: 0.7

In [68]:
print (f"Accuracy for GRU: {accuracy_score(Y_test, torch.argmax(model(X_test), dim=1)):.4f}")

Accuracy for GRU: 0.6603


#### The accuracy is better than simple RNN. It is now comparable to MLP with mean word2vec vectors and much higher than the MLP with concatenated word2vec vectors. Rest of the factors while comparing with MLP remain same as the RNN explanation above, but the vanishing gradient problem is reduced a bit and hence we see some better results than simple RNN.

## Task 5 (b) LSTM

#### same as RNN model but LSTM layer instead of RNN

In [69]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        output, hidden = self.rnn(text)
        output = output.mean(dim=1)
        fc_output = self.fc(output)
        return fc_output

model = LSTM(300, 20, 3)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
model.to(device)

LSTM(
  (rnn): LSTM(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=3, bias=True)
)

I have used the following hyperparameters: <br/>
learning rate = 0.01 <br/>
loss function = CrossEntropyLoss <br/>
optimizer = SGD <br/>
momentum = 0.9 <br/>
epochs = 30 <br/>
batch size = 32

In [70]:
model = train_model(model, optimizer, loss_fn, X_train, Y_train, X_test, Y_test, num_epochs=30, batch_size=32)

Epoch 1/30, Train Loss: 0.9286, Train Accuracy: 0.5455
Epoch 1/30, Test Loss: 0.9339, Test Accuracy: 0.5402
Epoch 2/30, Train Loss: 0.8642, Train Accuracy: 0.5926
Epoch 2/30, Test Loss: 0.8701, Test Accuracy: 0.5867
Epoch 3/30, Train Loss: 0.8247, Train Accuracy: 0.6250
Epoch 3/30, Test Loss: 0.8313, Test Accuracy: 0.6184
Epoch 4/30, Train Loss: 0.8333, Train Accuracy: 0.6178
Epoch 4/30, Test Loss: 0.8407, Test Accuracy: 0.6158
Epoch 5/30, Train Loss: 0.7945, Train Accuracy: 0.6382
Epoch 5/30, Test Loss: 0.8065, Test Accuracy: 0.6302
Epoch 6/30, Train Loss: 0.7854, Train Accuracy: 0.6469
Epoch 6/30, Test Loss: 0.7969, Test Accuracy: 0.6381
Epoch 7/30, Train Loss: 0.7695, Train Accuracy: 0.6532
Epoch 7/30, Test Loss: 0.7849, Test Accuracy: 0.6453
Epoch 8/30, Train Loss: 0.7646, Train Accuracy: 0.6533
Epoch 8/30, Test Loss: 0.7821, Test Accuracy: 0.6416
Epoch 9/30, Train Loss: 0.7702, Train Accuracy: 0.6531
Epoch 9/30, Test Loss: 0.7876, Test Accuracy: 0.6408
Epoch 10/30, Train Loss: 0.7

In [71]:
print (f"Accuracy for LSTM: {accuracy_score(Y_test, torch.argmax(model(X_test), dim=1)):.4f}")

Accuracy for LSTM: 0.6638


#### The accuracy is better than simple RNN and slightly better (almost same) than GRU. The comparison with MLP remains the same as RNN as well as the explanations. The vanishing gradient problem is reduced a bit compared to RNN and hence we see some better results than simple RNN. LSTM and GRU give almost similar results as they both handle vanishing gradient in their own way and both seem to performing almost equally in our case. (GRU and LSTM order varies from run to run).

GRU ~= LSTM > Simple RNN.

In this case, we see that LSTM performs the best, than GRU and finally simple RNN. This is because RNN faces the problem of vanisihing gradients which both LSTM and GRU try to handle in their own ways. GRU and LSTM both are equally good and their relative order is usually varying from run to run.